In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32).reshape(-1, 1, 28, 28)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [ ]:
class NeuralModel(nn.Module):
  def __init__(self, input_features, device):
    super().__init__()
    self.device = device
    self.criterion = nn.CrossEntropyLoss()

    self.features = nn.Sequential(
        nn.Conv2d(input_features, 32, kernel_size=3, padding='same'),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(32, 64, kernel_size=3, padding='same'),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(kernel_size=2, stride=2),

    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*7*7, 128),
        nn.ReLU(),
        nn.Dropout(0.4),

        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Dropout(0.4),

        nn.Linear(64, 10),
    )

    self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)


  def forward(self, X):
    X = self.features(X)
    X = self.classifier(X)
    return X

  def loss(self, X, y):
    y_pred = self.forward(X)
    return self.criterion(y_pred, y)

  def fit(self, train_loader, epochs):

    self.train()
    for epoch in range(epochs):
      total_loss = 0
      correct, total = 0, 0

      scaler = torch.amp.GradScaler('cuda')

      for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(self.device, non_blocking=True), y_batch.to(self.device, non_blocking=True)
        self.optimizer.zero_grad()

        with torch.amp.autocast('cuda'):
          outputs = self.forward(X_batch)
          loss = self.criterion(outputs, y_batch)

        scaler.scale(loss).backward()
        scaler.step(self.optimizer)
        scaler.update()

        total_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

      avg_loss = total_loss / len(train_loader)
      accuracy = 100 * correct / total
      print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f} - Accuracy: {accuracy:.2f}%")

  def evaluate(self, test_loader):
    self.eval()
    correct, total = 0, 0
    with torch.no_grad():
      for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(self.device), y_batch.to(self.device)

        outputs = self.forward(X_batch)
        _, predicted = torch.max(outputs, 1)

        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

    accuracy = 100 * correct / total
    print(F'Accuracy: {accuracy:.2f}%')

    return accuracy

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,)),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
])

In [ ]:
train_dataset = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 13.2MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 208kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.50MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 13.5MB/s]


In [ ]:
test_dataset = datasets.FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    pin_memory=True,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    pin_memory=True,
)

In [ ]:
images, labels = next(iter(train_loader))
print("Batch images shape:", images.shape)
print("Batch labels shape:", labels.shape)

Batch images shape: torch.Size([64, 1, 28, 28])
Batch labels shape: torch.Size([64])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralModel(1, device=device).to(device)

In [ ]:
model.fit(train_loader, epochs=10)

Epoch 1/10 - Loss: 0.9883 - Accuracy: 63.37%
Epoch 2/10 - Loss: 0.7578 - Accuracy: 72.31%
Epoch 3/10 - Loss: 0.6955 - Accuracy: 74.67%
Epoch 4/10 - Loss: 0.6594 - Accuracy: 76.21%
Epoch 5/10 - Loss: 0.6301 - Accuracy: 77.23%
Epoch 6/10 - Loss: 0.6096 - Accuracy: 78.39%
Epoch 7/10 - Loss: 0.5984 - Accuracy: 78.64%
Epoch 8/10 - Loss: 0.5796 - Accuracy: 78.92%
Epoch 9/10 - Loss: 0.5665 - Accuracy: 79.75%
Epoch 10/10 - Loss: 0.5534 - Accuracy: 80.22%


In [ ]:
model.evaluate(test_loader)

Accuracy: 81.50%


81.5